# Configuration de Cuda dans Google Colab

In [ ]:
!nvcc -V

In [ ]:
pip install nvcc4jupyter

## On vérifie que l'on est bien connecté au GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

## Chargement du plugin nvcc permettant de compiler/executer les programmes Cuda

In [ ]:
%load_ext nvcc4jupyter

## Un makefile est déjà à votre disposition pour compiler les programme du TP


### 1. Executez la cellule du Makefile

Le makefile a été modifié pour les programmes puisse s'exécuter avec la GPU premium.



In [ ]:
%%writefile Makefile
# Change the example variable to build a different source module (e.g. EXAMPLE=exercice01)
EXAMPLE=program

# Makefile variables
# Add extra targets to OBJ with space separator e.g. If there is as source file random.c then add random.o to OBJ)
# Add any additional dependancies (header files) to DEPS. e.g. if there is aheader file random.h required by your source modules then add this to DEPS.
CC=gcc
CFLAGS= -O3 -Wextra -fopenmp
NVCC=nvcc
NVCC_FLAGS= -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87
OBJ=$(EXAMPLE).o
DEPS=

# Build rule for object files ($@ is left hand side of rule, $< is first item from the right hand side of rule)
%.o : %.cu $(DEPS)
	$(NVCC) -c -o $@ $< $(NVCC_FLAGS) $(addprefix -Xcompiler ,$(CCFLAGS))

# Make example ($^ is all items from right hand side of the rule)
$(EXAMPLE) : $(OBJ)
	$(NVCC) -o $@ $^ $(NVCC_FLAGS) $(addprefix -Xcompiler ,$(CCFLAGS))

# PHONY prevents make from doing something with a filename called clean
.PHONY : clean
clean:
	rm -rf $(EXAMPLE) $(OBJ)

# TP2


## Fonctions utilitaires

utils.h est un header contenant des fonctions utilitaires qui seront utilisés par nos programmes

In [ ]:
%%writefile utils.h
#ifndef __UTILS_H__
#define __UTILS_H__
#include <stdio.h>

static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))


#define HANDLE_NULL( a ) {if (a == NULL) { \
                            printf( "Host memory failed in %s at line %d\n", \
                                    __FILE__, __LINE__ ); \
                            exit( EXIT_FAILURE );}}

template< typename T >
void swap( T& a, T& b ) {
    T t = a;
    a = b;
    b = t;
}


void* big_random_block( int size ) {
    unsigned char *data = (unsigned char*)malloc( size );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}

int* big_random_block_int( int size ) {
    int *data = (int*)malloc( size * sizeof(int) );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}


// a place for common kernels - starts here

__device__ unsigned char value( float n1, float n2, int hue ) {
    if (hue > 360)      hue -= 360;
    else if (hue < 0)   hue += 360;

    if (hue < 60)
        return (unsigned char)(255 * (n1 + (n2-n1)*hue/60));
    if (hue < 180)
        return (unsigned char)(255 * n2);
    if (hue < 240)
        return (unsigned char)(255 * (n1 + (n2-n1)*(240-hue)/60));
    return (unsigned char)(255 * n1);
}

__global__ void float_to_color( unsigned char *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset*4 + 0] = value( m1, m2, h+120 );
    optr[offset*4 + 1] = value( m1, m2, h );
    optr[offset*4 + 2] = value( m1, m2, h -120 );
    optr[offset*4 + 3] = 255;
}

__global__ void float_to_color( uchar4 *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset].x = value( m1, m2, h+120 );
    optr[offset].y = value( m1, m2, h );
    optr[offset].z = value( m1, m2, h -120 );
    optr[offset].w = 255;
}


#if _WIN32
    //Windows threads.
    #include <windows.h>

    typedef HANDLE CUTThread;
    typedef unsigned (WINAPI *CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC unsigned WINAPI
    #define  CUT_THREADEND return 0

#else
    //POSIX threads.
    #include <pthread.h>

    typedef pthread_t CUTThread;
    typedef void *(*CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC void
    #define  CUT_THREADEND
#endif

//Create thread.
CUTThread start_thread( CUT_THREADROUTINE, void *data );

//Wait for thread to finish.
void end_thread( CUTThread thread );

//Destroy thread.
void destroy_thread( CUTThread thread );

//Wait for multiple threads.
void wait_for_threads( const CUTThread *threads, int num );

#if _WIN32
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void *data){
        return CreateThread(NULL, 0, (LPTHREAD_START_ROUTINE)func, data, 0, NULL);
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        WaitForSingleObject(thread, INFINITE);
        CloseHandle(thread);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        TerminateThread(thread, 0);
        CloseHandle(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        WaitForMultipleObjects(num, threads, true, INFINITE);

        for(int i = 0; i < num; i++)
            CloseHandle(threads[i]);
    }

#else
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void * data){
        pthread_t thread;
        pthread_create(&thread, NULL, func, data);
        return thread;
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        pthread_join(thread, NULL);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        pthread_cancel(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        for(int i = 0; i < num; i++)
            end_thread( threads[i] );
    }

#endif




#endif  // __UTILS_H__

## 1. Addition vectorielle
Le but de cet exercice est de réaliser une addition vectorielle entre deux vecteurs de taille NxN, stockés dans la mémoire globale du GPU. Le résultat est stocké dans un troisième vecteur également de taille NxN, et lui aussi stocké dans la mémoire globale du GPU.

Le programme crée des blocs de threads de taille 2x2 et des grilles de blocs de la taille nécessaire pour couvrir tous les éléments des vecteurs.

Les vecteurs sont copiés depuis le CPU vers le GPU, puis le kernel est appelé pour effectuer l'addition vectorielle. Enfin, le résultat est renvoyé du GPU vers le CPU et affiché à l'écran.

### 1.1. Complétez l'instruction int size = ...
- La variable size représente la taille en octets d'un vecteur de dimension NxN.

### 1.2. Allouez la mémoire des vecteurs a, b et c dans la mémoire hôte (CPU)

### 1.3. Allouez la mémoire des vecteurs d_a, d_b et d_c dans la mémoire du périphérique (GPU)

### 1.4. Transférez les matrices a et b de la mémoire hôte (CPU) vers la mémoire du GPU à l'aide de la fonction cudaMemcpy

### 1.5. Complétez le kernel pour que chaque thread ai le bon index pour faire l'addition

Attention, le kernel lance une grille 2D de blocs 2D. Cette fois-ci les threads comme les blocs ont deux dimensions threadIdx.x et threadIdx.y et blockIdx.x blockIdx.y

La grille représente donc une matrice, mais a, b et c sont des vecteurs. Les étapes sont :

- Trouver l'index **col** représentant les colonnes de la matrice.
- Trouver l'index **row** représentant les lignes de la matrice.
- Trouver l'index global **index** permettant de remplir le vecteur **c** de taille NxN.

### 1.6 Transférez le vecteur d_c de la mémoire du GPU vers la mémoire hôte (CPU) à l'aide de la fonction cudaMemcpy

### 1.7 Libérez la mémoire de tous les vecteur à l'aide des fonctions free() et cudaFree()


In [ ]:
%%writefile program.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "utils.h"
#define N 4
#define BLOCK_SIZE 2

__global__ void vectorAdd(int *a, int *b, int *c) {

    // 1.5 Complete the kernel
    int col =
    int row =
    int index =

    if (col < N && row < N) {
        c[index] = a[index] + b[index];
    }
}

void printMatrix(int *a, int n) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            printf("%d\t", a[i * n + j]);
        }
        printf("\n");
    }
}

int main(void) {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;

    // 1.1 Complete the following instruction
    int size =

    // 1.2. Allocate memory on host
    a =
    b =
    c =

    // Initialize vectors on host
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            int index = i * N + j;
            a[index] = i;
            b[index] = j;
            c[index] = 0;
        }
    }

    // 1.3. Allocate memory on device
    HANDLE_ERROR(cudaMalloc());
    HANDLE_ERROR(cudaMalloc());
    HANDLE_ERROR(cudaMalloc());

    // Copy vectors from host to device
    HANDLE_ERROR(cudaMemcpy());
    HANDLE_ERROR(cudaMemcpy());

    // Define block and grid size
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 dimGrid((N + dimBlock.x - 1) / dimBlock.x, (N + dimBlock.y - 1) / dimBlock.y);

    // Call kernel function
    vectorAdd<<<dimGrid, dimBlock>>>(d_a, d_b, d_c);

    // 1.6 Copy result vector from device to host
    HANDLE_ERROR(cudaMemcpy());

    // print matrices
    printf("Matrix A:\n");
    printMatrix(a, N);

    printf("Matrix B:\n");
    printMatrix(b, N);

    printf("Matrix C:\n");
    printMatrix(c, N);

    // 1.7 Free memory on device and host

    return 0;
}

In [ ]:
!make

In [ ]:
!./program

## 2. Multiplication vectorielle.

Dans le TP1, vous avez utilisé un kernel multipliant les éléments de même index d'une matrice A et B.

Le but de cet exercice est de faire un programme qui effectue la multiplication matricielle de deux matrices carrées de taille N*N en utilisant CUDA. Les matrices sont initialisées sur le CPU et copiées sur le GPU avant d'être traitées par un kernel CUDA qui calcule les éléments de la matrice résultante en parallèle. Le résultat est ensuite copié sur le CPU et affiché à l'écran.

Ici on ne veux pas multiplier chaque élément d'une matrice A par l'élement de même indice de la matrice B. Un exemple de multiplication entre deux matrices vous est donné dans le lien suivant.

[Matmul image](https://drive.google.com/file/d/1XUP77D67Fm0BtITE7BtUdDCLrI8HHlRF/view?usp=share_link)

Brievement, pour multiplier deux matrices carrées A et B il faut :
Lancer une boucle pour itérer sur chaque élément de la ligne row de A et de la colonne col de B, et calculer le produit A[row][k] * B[k][col] et l'ajouter à la valeur courante de C[row][col].

### 1.1. Complétez l'instruction int size = ...
- La variable size représente la taille en octets d'un vecteur de dimension NxN.

### 1.2. Allouez la mémoire des vecteurs a, b et c dans la mémoire hôte (CPU)

### 1.3. Allouez la mémoire des vecteurs d_a, d_b et d_c dans la mémoire du périphérique (GPU)

### 1.4. Transférez les matrices a et b de la mémoire hôte (CPU) vers la mémoire du GPU à l'aide de la fonction cudaMemcpy

### 1.5. Complétez le kernel pour que chaque thread ai le bon index pour faire l'addition

Attention, le kernel lance une grille 2D de blocs 2D. Cette fois-ci les threads comme les blocs ont deux dimensions threadIdx.x et threadIdx.y et blockIdx.x blockIdx.y

La grille représente donc une matrice, mais a, b et c sont des vecteurs. Les étapes sont :

- Trouver l'index **col** représentant les colonnes de la matrice.
- Trouver l'index **row** représentant les lignes de la matrice.
- Trouver l'index global **index** permettant de remplir le vecteur **c** de taille NxN.
- Trouver L'index row_index représentant l'indice de l'élément i de la ligne row
- Trouver L'index col_index représentant l'indice de l'élément i de la colonne col

### 1.6 Transférez le vecteur d_c de la mémoire du GPU vers la mémoire hôte (CPU) à l'aide de la fonction cudaMemcpy

### 1.7 Libérez la mémoire de tous les vecteur à l'aide des fonctions free() et cudaFree()


In [ ]:
%%writefile program.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "utils.h"

#define N 4
#define BLOCK_SIZE 2

__global__ void matrixMultiplication(int *a, int *b, int *c, int n) {
    // 1.5 complete the kernel.
    int col =
    int row =

    index =

    if (col < n && row < n) {
        int sum = 0;
        for (int i = 0; i < n; i++) {
            row_index =
            col_index =
            sum += a[row_index] * b[col_index];
        }
        c[row * n + col] = sum;
    }
}

void initializeMatrix(int *a, int *b, int n) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            a[i * n + j] = i + j;
            b[i * n + j] = i - j;
        }
    }
}

void printMatrix(int *a, int n) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            printf("%d\t", a[i * n + j]);
        }
        printf("\n");
    }
}

int main() {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;

    // 1.1 Complete the following instruction
    int size =

    // 1.2 allocate host memory
    a =
    b =
    c =

    // initialize matrices
    initializeMatrix(a, b, N);

    // 1.3 allocate device memory
    HANDLE_ERROR(cudaMalloc());
    HANDLE_ERROR(cudaMalloc());
    HANDLE_ERROR(cudaMalloc());

    // 1.4 copy input data to device memory
    HANDLE_ERROR(cudaMemcpy());
    HANDLE_ERROR(cudaMemcpy());

    // launch kernel
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 dimGrid((N + dimBlock.x - 1) / dimBlock.x, (N + dimBlock.y - 1) / dimBlock.y);
    matrixMultiplication<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, N);

    // 1.6 copy result back to host memory
    HANDLE_ERROR(cudaMemcpy());

    // print matrices
    printf("Matrix A:\n");
    printMatrix(a, N);

    printf("Matrix B:\n");
    printMatrix(b, N);

    printf("Matrix C:\n");
    printMatrix(c, N);

    // 1.7 free memory

    return 0;
}

In [ ]:
!make

In [ ]:
!./program